In [1]:
import requests
import os
import pandas as pd

In [2]:
base_url = 'https://swapi.dev/api/'

In [3]:
tmp = requests.get(os.path.join(base_url, 'people')).json()

In [4]:
people = []
for i in range(1, 11):
    people.append(requests.get(os.path.join(base_url, 'people/{}'.format(i))).json())

In [5]:
def get_nested(d):
    urls = d['films']
    films = [requests.get(url).json() for url in urls]
    d['films']  = films
    return d
def get_titles(d):
    d['films'] = [d['films'][i]['title'] for i in range(len(d['films']))]
    return d
def get_birth_num(d):
    try:
        d['birth_year'] = float(d['birth_year'].replace("BBY", ""))
    except:
        d['birth_year'] = d['birth_year']
    return d

In [6]:
def convert_str(x):
    try:
        return int(x)
    except:
        return x

def to_num(data):
    for key in data:
        val = data[key]
        if isinstance(val, str):
            data[key] = convert_str(val)
        elif isinstance(val, dict):
            for k, v in val.items():
                if isinstance(v, str):
                    val[k] = convert_str(v)
        elif isinstance(val, list):
            for i, item in enumerate(val):
                if isinstance(item, str):
                    data[key][i] = convert_str(item)
                elif isinstance(item, dict):
                    for k, v in item.items():
                        if isinstance(v, str):
                            data[key][i][k] = convert_str(v)      
    return data

In [7]:
people_f = [to_num(get_birth_num(get_titles(get_nested(people[i])))) for i in range(len(people))]

In [8]:
df = pd.DataFrame(people_f).filter(['name', 'birth_year', 'films'])

In [9]:
df.columns = ['name', 'birth_year (BBY)', 'films']

In [12]:
(
    df[df['name']!='R5-D4'].
    groupby('name').
    sum().
    sort_values('birth_year (BBY)').
    filter(['name', 'birth_year (BBY)']).
    head(1)
)

,birth_year (BBY)
name,
Leia Organa,19.0


In [13]:
(
    df.
    groupby('name').
    sum().
    filter(['name', 'films'])
)

,films
name,
Beru Whitesun lars,"[A New Hope, Attack of the Clones, Revenge of ..."
Biggs Darklighter,[A New Hope]
C-3PO,"[A New Hope, The Empire Strikes Back, Return o..."
Darth Vader,"[A New Hope, The Empire Strikes Back, Return o..."
Leia Organa,"[A New Hope, The Empire Strikes Back, Return o..."
Luke Skywalker,"[A New Hope, The Empire Strikes Back, Return o..."
Obi-Wan Kenobi,"[A New Hope, The Empire Strikes Back, Return o..."
Owen Lars,"[A New Hope, Attack of the Clones, Revenge of ..."
R2-D2,"[A New Hope, The Empire Strikes Back, Return o..."
